In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

input_data_length = 54
model_num = input('Press model num :')

Made_X = np.load('Made_X/Made_X %s_%s.npy' % (input_data_length, model_num))
Made_Y = np.load('Made_X/Made_Y %s_%s.npy' % (input_data_length, model_num))

print(Made_X.shape)
print(Made_Y.shape)
print(np.sum(Made_Y))

# pyplot.figure(figsize=(10,5))
# pyplot.plot(Made_Y)
# pyplot.show()

row = Made_X.shape[1]
col = Made_X.shape[2]

total_len = len(Made_X)
train_len = int(total_len * 0.7)
val_len = int(total_len * 0.15)
test_len = total_len - (train_len + val_len)

X_train = Made_X[:train_len].astype('float32').reshape(-1, input_data_length, col, 1)
X_val = Made_X[train_len:train_len + val_len].astype('float32').reshape(-1, input_data_length, col, 1)
X_test = Made_X[train_len + val_len:].astype('float32').reshape(-1, input_data_length, col, 1)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)


from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator 

Y_train = Made_Y[:train_len].astype('float32')
Y_val = Made_Y[train_len:train_len + val_len].astype('float32')
Y_test = Made_Y[train_len + val_len:].astype('float32')
num_classes = 3
Y_train = np_utils.to_categorical(Y_train, num_classes)
Y_val = np_utils.to_categorical(Y_val, num_classes)
Y_test = np_utils.to_categorical(Y_test, num_classes)
print(Y_train.shape)
print(Y_val.shape)
print(Y_test.shape)

datagen = ImageDataGenerator( 
    rotation_range = 60,
    horizontal_flip = True,
    width_shift_range=0.6,
    height_shift_range=0.6,
    fill_mode = 'nearest'
    )

testgen = ImageDataGenerator( 
    )
datagen.fit(X_train)
batch_size = 128

# for X_batch, _ in datagen.flow(X_train, Y_train, batch_size=9):
#     for i in range(0, 9): 
#         pyplot.axis('off') 
#         pyplot.subplot(330 + 1 + i) 
#         pyplot.imshow(X_batch[i].reshape(input_data_length, col), cmap=pyplot.get_cmap('gray'))
#     pyplot.axis('off') 
#     pyplot.show() 
#     break
    
    
train_flow = datagen.flow(X_train, Y_train, batch_size=batch_size) 
val_flow = testgen.flow(X_val, Y_val, batch_size=batch_size) 
test_flow = testgen.flow(X_test, Y_test, batch_size=batch_size)


from keras.utils import plot_model
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import LSTM, TimeDistributed, Input, Dense, Flatten, Dropout, BatchNormalization, Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D, MaxPooling1D
from keras.layers.merge import concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from sklearn.metrics import confusion_matrix

def build_network(keep_prob=0.5, optimizer='adam'):
    # first input model
    input_shape=(input_data_length, col, 1)
    visible = Input(shape=input_shape, name='input')
    conv1_fit = 100
    conv2_fit = 100
    conv3_fit = 128
    # conv4_fit = 256
    # conv5_fit = 256

    #the 1-st block
    conv1_1 = Conv2D(conv1_fit, kernel_size=3, activation='relu', padding='same', name = 'conv1_1')(visible)
    conv1_1 = BatchNormalization()(conv1_1)
    # conv1_2 = Conv2D(conv1_fit, kernel_size=3, activation='relu', padding='same', name = 'conv1_2')(conv1_1)
    # conv1_2 = BatchNormalization()(conv1_2)
    pool1_1 = MaxPooling2D(pool_size=(2,2), name = 'pool1_1')(conv1_1)
    # drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)

    #the 2-nd block
    conv2_1 = Conv2D(conv2_fit, kernel_size=3, activation='relu', padding='same', name = 'conv2_1')(pool1_1)
    conv2_1 = BatchNormalization()(conv2_1)
    # conv2_2 = Conv2D(conv2_fit, kernel_size=3, activation='relu', padding='same', name = 'conv2_2')(conv2_1)
    # conv2_2 = BatchNormalization()(conv2_2)
    # conv2_3 = Conv2D(conv2_fit, kernel_size=3, activation='relu', padding='same', name = 'conv2_3')(conv2_2)
    # conv2_3 = BatchNormalization()(conv2_3)
    pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_1)
    drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)

    #Flatten and output
    flatten = Flatten(name = 'flatten')(drop2_1)
    dense = Dense(100, activation='relu', name='dense')(flatten)
    output = Dense(num_classes, activation='softmax', name = 'output')(dense)

    # create model 
    model = Model(inputs =visible, outputs = output)
    # summary layers
    print(model.summary())
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    
    return model

def create_hyperparams():
    batches = np.arange(10, 151, 20)
    optimizers = ['adam']
#     optimizers = ['rmsprop', 'adam', 'adadelta']

    dropout = np.linspace(0.1, 0.5, 5)
    epochs = [100]
    return {'batch_size': batches, 'optimizer': optimizers, 'keep_prob': dropout, 
           'epochs': epochs}

from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=build_network, verbose=1)

hyperparams = create_hyperparams()

from sklearn.model_selection import RandomizedSearchCV
search = RandomizedSearchCV(estimator=model,
                            param_distributions=hyperparams,
                            n_iter=10, n_jobs=1, cv=3, verbose=1)
from keras.callbacks import EarlyStopping
checkpoint3 = EarlyStopping(monitor='val_loss', patience=15)
callbacks_list = [checkpoint3]

search.fit(X_train, Y_train, validation_data=(X_val, Y_val), callbacks=callbacks_list)
print(search.best_params_)


Using TensorFlow backend.
/home/lab23/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lab23/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lab23/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lab23/.local/lib/python3.6/site-packages/tensorflow/python/fra

Press model num :21
(455547, 54, 6)
(455547, 1)
149637.0
(318882, 54, 6, 1)
(68332, 54, 6, 1)
(68333, 54, 6, 1)
(318882, 3)
(68332, 3)
(68333, 3)
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 54, 6, 1)          0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 54, 6, 100)        1000      
_________________________________________________________________
batch_normalization_1 (Batch (None, 54, 6, 100)        400       
_________________________________________________________________
pool1_1 (MaxPooling2D)       (None, 27, 3, 100)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 27, 3, 100)        90100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 27, 3, 100)        400       
_________________________________________________________________
pool2_1 (MaxPooling2D)       (None, 13, 1, 100)        0  

Epoch 12/100
212588/212588 [==============================] - 12s 58us/step - loss: 0.6499 - acc: 0.7819 - val_loss: 0.6291 - val_acc: 0.8010
Epoch 13/100
212588/212588 [==============================] - 12s 58us/step - loss: 0.6481 - acc: 0.7822 - val_loss: 0.6302 - val_acc: 0.8010
Epoch 14/100
212588/212588 [==============================] - 12s 58us/step - loss: 0.6465 - acc: 0.7824 - val_loss: 0.6298 - val_acc: 0.8011
Epoch 15/100
212588/212588 [==============================] - 12s 58us/step - loss: 0.6444 - acc: 0.7826 - val_loss: 0.6359 - val_acc: 0.7979
Epoch 16/100
212588/212588 [==============================] - 12s 58us/step - loss: 0.6421 - acc: 0.7829 - val_loss: 0.6315 - val_acc: 0.8005
Epoch 17/100
212588/212588 [==============================] - 12s 58us/step - loss: 0.6407 - acc: 0.7834 - val_loss: 0.6397 - val_acc: 0.7985
Epoch 18/100
212588/212588 [==============================] - 12s 58us/step - loss: 0.6375 - acc: 0.7841 - val_loss: 0.6323 - val_acc: 0.7995
Epoch 

KeyboardInterrupt: 